<a href="https://colab.research.google.com/github/HazemHassine/Machine-Learning-Components/blob/master/Linear_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook, the algorithms implemented are:
1. hinge Loss
2. Cost Gradient
3. Stochastic Gradient Descent

In [5]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

def compute_cost(weights, X, Y):
  N = X.shape[0]
  distances = 1 - Y* np.dot(W , X)
  distances[distances < 0] = 0 # equal to max(0, distance)
  hinge_loss = reguralization_strength * np.sum(distances) / N

  cost = 1/2 * np.dot(W,W) + hinge_loss # np.dot(W,W) gets the normal length of W vector

  return cost

In [2]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array for compatability

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
            # the regularization strength usually denoted as lambda, or 1/Ci 
            # this variable dsecribes variance of the margin's width
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [3]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights